# All Conference Transfers

In [1]:
import json
import core_constants as cc
import functions as fx
import pandas as pd
import sqlite3 as sql
import recordlinkage
from recordlinkage.index import Full
import queries

## Build out DataFrames

In [17]:
ranked = True
query = queries.get_query_Transfers247(ranked)
df_247 = (fx.connDBAndReturnDF(query)).set_index('IDYR')
df_247.index.name = '247_IDYR'
df_247

,ID,PlayerName,Year,College,HighSchool,City,State,KeyPositionGroup,StandardizedPosition,Height,Weight
247_IDYR,,,,,,,,,,,
paulthurston_nebraska_2012,paulthurston_nebraska,paulthurston,2012,nebraska,Arvada West,Arvada,CO,1,OL,77.0,275.0
gregmcmullen_nebraska_2012,gregmcmullen_nebraska,gregmcmullen,2012,nebraska,Archbishop Hoban,Akron,OH,2,DE,77.0,255.0
jordanwesterkamp_nebraska_2012,jordanwesterkamp_nebraska,jordanwesterkamp,2012,nebraska,Montini Catholic,Lombard,IL,1,WR,73.0,190.0
imanicross_nebraska_2012,imanicross_nebraska,imanicross,2012,nebraska,North Hall,Gainesville,GA,1,RB,73.0,222.0
tommyarmstrong_nebraska_2012,tommyarmstrong_nebraska,tommyarmstrong,2012,nebraska,Steele,Cibolo,TX,1,QB,73.0,200.0
...,...,...,...,...,...,...,...,...,...,...,...
dondrellharris_troy_2013,dondrellharris_troy,dondrellharris,2013,troy,Rutherford,Panama City,FL,1,QB,72.0,182.0
jordanchunn_troy_2013,jordanchunn_troy,jordanchunn,2013,troy,Madison County,Gurley,AL,1,FB,72.0,211.0
bryanslater_troy_2013,bryanslater_troy,bryanslater,2013,troy,St. Paul's Episcopal,Mobile,AL,2,DE,74.0,255.0


In [29]:
limit = False
year = '2004'
query = queries.get_query_UnlinkedAllConference(limit,year)
df_ac = (fx.connDBAndReturnDF(query)).set_index('ID')
df_ac.index.name = 'AllConf_ID'
df_ac

,PlayerName,Year,College,KeyPositionGroup,StandardizedPosition
AllConf_ID,,,,,
odafeoweh_pennstate,odafeoweh,2020,pennstate,2,DL
ekuleota_northwestern,ekuleota,2020,northwestern,2,DL
miketaylor_wisconsin,miketaylor,2011,wisconsin,2,LB
marcuscoleman_wisconsin,marcuscoleman,2007,wisconsin,1,OL
jeremyleman_illinois,jeremyleman,2007,illinois,2,LB
...,...,...,...,...,...
hoostmarsh_wyoming,hoostmarsh,2004,wyoming,4,ATH
zachmorris_wyoming,zachmorris,2004,wyoming,2,DL
jovonbouknight_wyoming,jovonbouknight,2005,wyoming,1,WR


### NOTES

> You don't have position data, so you are doing the best you can here.  Blocking on exact name match isn't great, but you'll have to change how you collect from wikipedia to change this - cause you'll need to fetch Position data so you can derive position group and block on that.

In [ ]:
#indexer = recordlinkage.Index()
#indexer = recordlinkage.BlockIndex(on=['KeyPositionGroup'])

In [19]:
#To avoid the Athlete issue, I'm not doing a full index instead of blocking
indexer = recordlinkage.Index()
indexer.add(Full())

In [20]:
pairs = indexer.index(df_ac, df_247)

In [21]:
from functions import YearOther

sumFields = []
c = recordlinkage.Compare()
c.string('PlayerName', 'PlayerName', method='damerau_levenshtein', label='PlayerName')
sumFields.append('PlayerName')
#c.exact('StandardizedPosition', 'StandardizedPosition', label='StandardizedPosition')
#sumFields.append('StandardizedPosition')
#c.exact('KeyPositionGroup', 'KeyPositionGroup', label='KeyPositionGroup')
#sumFields.append('KeyPositionGroup')
c.add(YearOther('Year', 'Year', label='Year'))
sumFields.append('Year')

In [22]:
features = c.compute(pairs, df_ac, df_247)

In [23]:
sum = 0
for field in sumFields:
    sum = sum + features[field]

features['Sum'] = sum / len(sumFields)

features = features[features['Year'] == 1.0]

features

PlayerName  Year  \
AllConf_ID          247_IDYR                                                  
odafeoweh_pennstate sampsonjames_indiana_2019                0.166667   1.0   
                    beaurobbins_indiana_2019                 0.181818   1.0   
                    cameronwilliams_indiana_2019             0.200000   1.0   
                    joshsanguinetti_indiana_2019             0.200000   1.0   
                    tiawanmullen_indiana_2019                0.166667   1.0   
...                                                               ...   ...   
warddobbs_wyoming   jessebowman_mississippistate_2007        0.090909   1.0   
                    paulnichols_arizona_2005                 0.181818   1.0   
                    geneanderson_southernmississippi_2007    0.166667   1.0   
                    cammontgomery_northtexas_2007            0.153846   1.0   
                    jonathanharris_sanjosestate_2007         0.142857   1.0   

                                                                Sum  
AllConf_ID          247_IDYR                                         
odafeoweh_pennstate sampsonjames_indiana_2019              0.583333  
                    beaurobbins_indiana_2019               0.590909  
                    cameronwilliams_indiana_2019           0.600000  
                    joshsanguinetti_indiana_2019           0.600000  
                    tiawanmullen_indiana_2019              0.583333  
...                                                             ...  
warddobbs_wyoming   jessebowman_mississippistate_2007      0.545455  
                    paulnichols_arizona_2005               0.590909  
                    geneanderson_southernmississippi_2007  0.583333  
                    cammontgomery_northtexas_2007          0.576923  
                    jonathanharris_sanjosestate_2007       0.571429  

[1301837 rows x 3 columns]

In [24]:
features.insert(0, 'sourceID', features.index.get_level_values(0))
features.insert(1, 'targetID', features.index.get_level_values(1))

filteredList = []
for idx, data in features.groupby(level=0):
    data = data.loc[data['Sum'].idxmax()]
    #.929
    if (data['Sum'] > .9):
        filteredList.append(data)
dfFinal = pd.DataFrame()
dfFinal = dfFinal.append(filteredList)
dfFinal.to_csv("resultsAllConferenceTransfer.csv")

In [25]:
dfFinal

,sourceID,targetID,PlayerName,Year,Sum
"(alexjohnson_sandiegostate, kalebjohnson_rutgers_2011)",alexjohnson_sandiegostate,kalebjohnson_rutgers_2011,0.833333,1.0,0.916667
"(andraezwilliams_lsu, andrewwilliams_auburn_2014)",andraezwilliams_lsu,andrewwilliams_auburn_2014,0.866667,1.0,0.933333
"(anthonyamos_middletennesseestate, anthonyalosi_pennstate_2011)",anthonyamos_middletennesseestate,anthonyalosi_pennstate_2011,0.833333,1.0,0.916667
"(anthonykent_kentstate, anthonywint_floridainternational_2014)",anthonykent_kentstate,anthonywint_floridainternational_2014,0.818182,1.0,0.909091
"(benardrickmckinney_missst, benardrickmckinney_mississippistate_2011)",benardrickmckinney_missst,benardrickmckinney_mississippistate_2011,1.000000,1.0,1.000000
"(calvinturner_hawaii, collinturner_louisianamonroe_2017)",calvinturner_hawaii,collinturner_louisianamonroe_2017,0.833333,1.0,0.916667
"(camerondantzler_missst, camerondantzler_mississippistate_2016)",camerondantzler_missst,camerondantzler_mississippistate_2016,1.000000,1.0,1.000000
"(charliejones_iowa, charliejones_buffalo_2017)",charliejones_iowa,charliejones_buffalo_2017,1.000000,1.0,1.000000
"(cjsanders_smu, cjsanders_notredame_2015)",cjsanders_smu,cjsanders_notredame_2015,1.000000,1.0,1.000000
"(connorculp_nebraska, connorculp_lsu_2016)",connorculp_nebraska,connorculp_lsu_2016,1.000000,1.0,1.000000


In [26]:
fuzzyMI = pd.MultiIndex.from_frame(dfFinal)
recordlinkage.write_annotation_file(
    "../Annotations/Annotations/annotation_ACTransfersRanked.json",
    fuzzyMI[0:160],
    df_ac,
    df_247,
    dataset_a_name="All Conference",
    dataset_b_name="Master"
)

In [27]:
annotation = recordlinkage.read_annotation_file("..//Annotations//Results//result-acranked.json")
try:
    annotation_dict = (annotation.links).to_flat_index()
except Exception as e:
    print(e)

In [28]:
for record in annotation_dict:
    #MAKE SURE YOU UPDATE THE THIRD VALUE TO THE CORRECT KEYDATASET!!
    Values = [record[0], record[1], 4, 1, 1, 1,1]
    query = '''INSERT INTO RecordLinks(MasterID, TargetID, KeyDataSet, TargetKeyDataSet, KeyLinkType, LinkConfidence, Transfer)
        VALUES (?,?,?,?,?,?,?)'''
    
    conn = sql.connect(cc.databaseName)
    c = conn.cursor()
    
    c.execute(query, Values)
    conn.commit()
    
conn.close()